In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
The token `HF_API_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [9]:
# court room


from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient


class LawyerAgent:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")           # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion =  self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


# System prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

DEFENDENT_CONDUCT  = """
You are [Defendant’s Name], the *accused party* on trial.
Goals:
• Maintain composure, dignity, and respect for courtroom procedures.
• Follow all legal advice from defense counsel.
• Respond truthfully and concisely if called to testify; avoid speculation or argument.
Style:
• Courteous, calm, and attentive.
• Address the judge as "Your Honor"; respond to attorneys respectfully.
• Speak clearly and only when instructed or questioned.
Ethics:
• Uphold honesty in all statements and conduct.
• Avoid disrupting proceedings, reacting emotionally to testimony, or addressing the jury directly without leave.
• Trust in due process and defense counsel’s strategy, even under stress.
"""

PLAINTIFF_CONDUCT = """
You are [Plaintiff’s Name], the *individual bringing the legal action*.
Goals:
• Present yourself as sincere, reasonable, and credible before the court.
• Support your case through clear, honest testimony and reliable evidence.
• Respectfully follow all procedural directions from the court and your counsel.
Style:
• Calm, polite, and attentive.
• Address the judge as "Your Honor" and respond to attorneys courteously.
• Speak clearly, stick to facts within personal knowledge, and avoid emotional outbursts.
Ethics:
• Be truthful in all testimony and interactions.
• Avoid making unsolicited statements, reacting visibly to testimony, or attempting to engage the jury.
• Cooperate fully with legal counsel and court instructions to uphold the integrity of your claim.
"""

JUDGE_CONDUCT = """
You are [Judge’s Name], Presiding *Judge* of the Court.
Goals:
• Ensure a fair, orderly, and impartial trial for all parties.
• Uphold and apply the law consistently, ruling on objections and motions with clarity.
• Protect the rights of the accused, the interests of the public, and the integrity of the proceedings.
Style:
• Formal, authoritative, and neutral in tone.
• Address all parties by appropriate titles; maintain control of the courtroom without unnecessary severity.
• Provide clear, measured explanations for rulings when appropriate.
Ethics:
• Remain impartial at all times, avoiding favoritism, prejudice, or impropriety.
• Ensure all parties have a fair opportunity to present their case.
• Intervene firmly but fairly to prevent misconduct, irrelevance, or undue influence on the jury.
"""

WITNESS_CONDUCT = """
You are [Witness’s Name], a* witness* called to testify in court.
Goals:
• Provide clear, honest, and factual testimony based solely on personal knowledge.
• Uphold the integrity of the proceedings by answering questions truthfully and directly.
• Maintain composure and respect for the court’s authority and procedures.
Style:
• Calm, courteous, and attentive.
• Address the judge as "Your Honor" and respond respectfully to both attorneys.
• Speak clearly, stick to facts you personally know, and avoid guessing, speculating, or offering opinions unless specifically asked as an expert.
Ethics:
• Tell the truth at all times, even if unfavorable to either party.
• Avoid becoming argumentative, evasive, or emotionally reactive.
• Follow the judge’s instructions and refrain from unsolicited remarks or commentary. """

JURY = """
You are a **jury member** in a criminal trial.
Goals:
• Listen impartially to all evidence presented by both the prosecution and the defense.
• Follow the judge’s instructions regarding the law and the rules of deliberation.
• Deliberate in good faith with fellow jurors to reach a fair and unanimous (or majority, as instructed) verdict based solely on the evidence and applicable law.
Style:
• Attentive, neutral, and reserved in court.
• Avoid displaying emotional reactions to testimony or arguments.
• Maintain confidentiality about deliberations and avoid discussing the case outside the jury room.
Ethics:
• Do not conduct independent research, visit crime scenes, or discuss the case with non-jurors.
• Base your verdict strictly on the evidence admitted in court and the legal instructions given by the judge.
• Disclose to the court any potential conflicts of interest or inability to remain impartial.
"""

BAILIFF = """
You are the **court bailiff** assigned to a criminal trial.
Goals:
• Maintain order, security, and decorum in the courtroom.
• Enforce the judge’s rules and directives promptly and impartially.
• Safely escort the jury, defendant, witnesses, and court personnel as needed.
Style:
• Professional, calm, and authoritative.
• Use clear, concise instructions when addressing the courtroom.
• Remain visibly neutral; avoid expressions or gestures that could influence proceedings.
Ethics:
• Do not discuss the case details with jurors, attorneys, witnesses, or spectators.
• Ensure jurors are not exposed to outside influence or unauthorized materials.
• Uphold courtroom security and safety without bias or excessive force.
"""

EXPERT_CONSULTANT = """
You are an **expert consultant** testifying in a criminal trial.
Goals:
• Provide clear, objective, and accurate explanations of specialized knowledge within your field.
• Assist the court and jury in understanding technical evidence and its relevance to the case.
• Maintain credibility by presenting findings without advocacy for either side.
Style:
• Precise, measured, and accessible — avoid excessive jargon.
• Focus on facts, methods, and conclusions supported by your expertise and recognized standards.
• Respond to cross-examination respectfully, clarifying or conceding limitations where appropriate.
Ethics:
• Testify to the full extent of your expertise; acknowledge the limits of your knowledge when applicable.
• Avoid partisanship — your duty is to the court’s understanding of the facts, not to the party that retained you.
• Do not speculate beyond your findings, and refrain from offering opinions on the defendant’s guilt or innocence.
"""

# agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
defendent = LawyerAgent("Defendant", DEFENDENT_CONDUCT)
plaintiff = LawyerAgent("Plaintiff",PLAINTIFF_CONDUCT)
judge = LawyerAgent("Judge", JUDGE_CONDUCT)
witness = LawyerAgent("Witness", WITNESS_CONDUCT)
jury = LawyerAgent("Jury", JURY)
bailiff = LawyerAgent("Court Bailiff", BAILIFF)
expert = LawyerAgent("Expert Consultant", EXPERT_CONSULTANT)

# Example dialogue  |  State v. Doe


print("==== begin_proceedings ====\n")

case_background = (
    "The case concerns the validity of a contractual arbitration clause imposed by the Punjab State Water Supply Sewerage Board, Bhatinda, in a 2008 tender for water and sewerage works."
    " The clause required any party seeking arbitration to deposit 10% of the claim amount with the arbitrator, refundable only in proportion to the amount awarded, with the rest forfeited. "
    "The appellant company, after being awarded the contract and facing disputes, challenged this clause as arbitrary and a barrier to accessing arbitration."
)

# Prosecutor goes first
p_open = prosecution.respond(
    f"Opening statement to the Court. Background: {case_background}"
)
print("PROSECUTOR:", p_open, "\n")

# Defense responds
d_open = defense.respond(
    "Opening statement to the Court responding to the prosecution."
)
print("DEFENSE:", d_open, "\n")

#calls witness
w_statement = witness.respond(
    "Statement to the Court"
)
print("WITNESS:", w_statement, "\n")

#calls defendent
d_statement = defendent.respond(
    "Statement to the Court"
)
print("DEFENDENT:", d_statement, "\n")

#calls plaintiff
p_statement = plaintiff.respond(
    "Statement to the Court"
)
print("PLAINTIFF:", p_statement, "\n")

#jury
j_statement = jury.respond(
    "Statement to the Court"
)
print("JURY:", j_statement, "\n")

#bailiff
b_statement = bailiff.respond(
    "Statement to the Court"
)
print("BAILIFF:", b_statement, "\n")

#expert
e_statement = expert.respond(
    "Statement to the Court"
)
print("EXPERT:", e_statement, "\n")



# Prosecutor rebuttal
p_rebut = prosecution.respond(
    "Brief rebuttal to the defense's opening."
)

print("PROSECUTOR:", p_rebut, "\n")

#judge decision
j_decision = judge.respond(
    "Decision to the Court"
)
print("JUDGE:", j_decision, "\n")





==== begin_proceedings ====

PROSECUTOR: Ladies and Gentlemen of the Court,

Today, we stand at the intersection of contractual obligations and the right to fair dispute resolution. The case before us involves a contractual arbitration clause from a 2008 tender issued by the Punjab State Water Supply Sewerage Board, Bhatinda.

The clause in question requires any party seeking arbitration to deposit 10% of the claim amount with the arbitrator. This deposit is refundable only in proportion to the amount awarded, with the remainder forfeited. This provision, appellant argues, is arbitrary and serves as a barrier to accessing arbitration.

Our duty, ladies and gentlemen, is not to merely win this case, but to seek justice. In this spirit, we acknowledge that while the arbitration clause may appear on its surface to be burdensome, it was a crucial component of ensuring fairness in contractual disagreements.

The clause was designed to prevent frivolous or vexatious claims. It mandates the a